In [1]:
import pandas as pd
import numpy as np

# Загрузка данных

In [2]:
company = pd.read_excel('./Компании.xlsx') #загружаю основной набор данных

In [3]:
company.head()

,Наименование,Дата регистрации,Код статистики,Отрасль,Форма собственности,Код причины постановки на учет,Органы власти и государственного управления,СПАРК код,Размер компании,"2018, Среднесписочная численность работников",...,"2019, Сальдо денежных потоков за отчетный период, RUB","2019, Поступление денежных средств всего, RUB","2019, Платежей всего, RUB","2019, Прибыль от приносящей доход деятельности, RUB","2019, Всего поступило средств, RUB","2019, Расходы на целевые мероприятия, RUB","2019, Расходы на содержание аппарата управления, RUB","2019, Приобретение основных средств, инвентаря и иного имущества, RUB","2019, Всего использовано средств, RUB","2019, Остаток средств на конец отчетного года, RUB"
0,"033, ООО",2003-10-14,71115178.0,Производство,Частная собственность,390601001.0,"Организации, учрежденные юридическими лицами и...",4283790,Микропредприятия,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1 БЭЙК, ООО",2019-10-28,42006784.0,Производство,Частная собственность,391401001.0,"Организации, учрежденные юридическими лицами и...",12390280,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"1-Я ЛИФТОВАЯ, ООО",2014-11-11,35394691.0,Производство,Частная собственность,390601001.0,"Организации, учрежденные юридическими лицами и...",9603036,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"12 СТУЛЬЕВ, ООО",2006-03-29,94037570.0,Производство,Частная собственность,390601001.0,"Организации, учрежденные юридическими лицами и...",5298659,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"2020-ЛБК, ООО",2009-07-28,60596246.0,Производство,Частная собственность,390601001.0,"Организации, учрежденные юридическими лицами и...",7128967,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
bank_guarantees = pd.read_excel('./Банковские гарантии.xlsx') #данные о наличии банковских гарантий

In [5]:
leasing = pd.read_excel('./Лизинг.xlsx') #данные о наличии действующих договоров лизинга

In [6]:
security_loan = pd.read_excel('./Залоги.xlsx') #данные о заргестированных залогах(признак нличия залогового кредита)

In [7]:
data = pd.merge(company, bank_guarantees, how='left') #объединяю исседуемые данные с данными  гарантиях

In [8]:
data = pd.merge(data, leasing, how='left') #объединяю исседуемые данные с данными  лизинге

In [9]:
data = pd.merge(data, security_loan, how='left') #объединяю исседуемые данные с данными  кредитвании

# Обработка пропусков

In [10]:
pd.isnull(data).sum()
#после объединения данных получил большое количество пропусков в столбцах которые планирую использовать как данные для предсказания

Наименование                                              0
Дата регистрации                                         17
Код статистики                                           57
Отрасль                                                   0
Форма собственности                                      41
                                                      ...  
2019, Всего использовано средств, RUB                 12213
2019, Остаток средств на конец отчетного года, RUB    12275
БГ                                                    12160
Лизинг                                                11997
Залоги                                                12113
Length: 148, dtype: int64

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12500 entries, 0 to 12499
Columns: 148 entries, Наименование to Залоги
dtypes: datetime64[ns](1), float64(139), int64(1), object(7)
memory usage: 14.2+ MB


In [12]:
parth1 = data[['СПАРК код', 'БГ', 'Лизинг', 'Залоги']].fillna(0) #не разобрался как заполненить пропуски в series(хочу заполнить пропуски только в определенных столбцах)
#снова разбил данные на 2 dataFrame и обраотал одну из частей

In [13]:
parth2 = data.copy() #копирую исходные данные во вторую часть(для того чтобы избежать дублирования столбцов)

In [14]:
parth2.drop(['БГ', 'Лизинг', 'Залоги'], axis='columns', inplace=True) #удаляю столбцы с пропусками

In [15]:
pd.isnull(parth2).sum() #проверка на наличие пропусков

Наименование                                                                 0
Дата регистрации                                                            17
Код статистики                                                              57
Отрасль                                                                      0
Форма собственности                                                         41
                                                                         ...  
2019, Расходы на целевые мероприятия, RUB                                12350
2019, Расходы на содержание аппарата управления, RUB                     12293
2019, Приобретение основных средств, инвентаря и иного имущества, RUB    12447
2019, Всего использовано средств, RUB                                    12213
2019, Остаток средств на конец отчетного года, RUB                       12275
Length: 145, dtype: int64

In [16]:
data = pd.merge(parth1, parth2, how='left') #сноваобъединяю преобразованные данные

In [17]:
pd.isnull(data).sum()

СПАРК код                                                                    0
БГ                                                                           0
Лизинг                                                                       0
Залоги                                                                       0
Наименование                                                                 0
                                                                         ...  
2019, Расходы на целевые мероприятия, RUB                                12350
2019, Расходы на содержание аппарата управления, RUB                     12293
2019, Приобретение основных средств, инвентаря и иного имущества, RUB    12447
2019, Всего использовано средств, RUB                                    12213
2019, Остаток средств на конец отчетного года, RUB                       12275
Length: 148, dtype: int64

In [19]:
pd.isnull(data).sum().nlargest(60) #выявляем столбцы с самым болшим количеством пропусков

2019, Поступления от выпуска акций, увеличения долей участия, RUB                             12499
2019, Поступления от выпуска облигаций, векселей и других долговых ценных бумаг и др., RUB    12498
2019, Поступления от перепродажи финансовых вложений, RUB                                     12497
2019, Приобретение акций (долей участия), RUB                                                 12497
2018, Платежи по процентам, RUB                                                               12496
2019, Выкуп акций (долей) у собственников, RUB                                                12496
2019, Платежи по процентам, RUB                                                               12495
2019, Продажа акций других организаций, RUB                                                   12493
2019, Собственные акции, выкупленные у акционеров, RUB                                        12492
2019, Доходы от участия в других организациях, RUB                                            12491


In [21]:
data.describe()

,СПАРК код,БГ,Лизинг,Залоги,Код статистики,Код причины постановки на учет,"2017, Доходы, RUB","2018, Доходы, RUB","2017, Расходы, RUB","2018, Расходы, RUB",...,"2019, Сальдо денежных потоков за отчетный период, RUB","2019, Поступление денежных средств всего, RUB","2019, Платежей всего, RUB","2019, Прибыль от приносящей доход деятельности, RUB","2019, Всего поступило средств, RUB","2019, Расходы на целевые мероприятия, RUB","2019, Расходы на содержание аппарата управления, RUB","2019, Приобретение основных средств, инвентаря и иного имущества, RUB","2019, Всего использовано средств, RUB","2019, Остаток средств на конец отчетного года, RUB"
count,1.250000e+04,12500.000000,12500.000000,12500.000000,1.244300e+04,1.240000e+04,6.529000e+03,6.723000e+03,6.529000e+03,6.723000e+03,...,7.700000e+02,7.760000e+02,7.990000e+02,4.900000e+01,2.830000e+02,1.500000e+02,2.070000e+02,5.300000e+01,2.870000e+02,2.250000e+02
mean,7.280235e+06,0.027200,0.040240,0.030960,4.740837e+07,3.909506e+08,3.696273e+07,6.765525e+07,3.527823e+07,6.378047e+07,...,-3.315117e+05,7.201096e+08,5.975395e+08,1.085373e+07,1.430393e+07,8.154213e+06,3.954333e+06,3.448113e+05,1.364463e+07,2.914098e+06
std,3.434821e+06,0.162672,0.196529,0.173216,2.765877e+07,5.580382e+06,2.542989e+08,7.662991e+08,2.496199e+08,7.547269e+08,...,1.215834e+08,4.558372e+09,4.556149e+09,5.143724e+07,5.945618e+07,3.095065e+07,2.245927e+07,2.326990e+06,5.885619e+07,4.315045e+07
min,1.536000e+03,0.000000,0.000000,0.000000,4.716000e+04,2.315010e+08,0.000000e+00,-1.194900e+07,0.000000e+00,0.000000e+00,...,-2.101677e+09,-4.380000e+05,-2.693985e+10,-2.091000e+06,-2.041000e+06,-7.919000e+06,-6.029000e+06,-1.403200e+07,-2.922800e+07,-4.208010e+08
25%,4.867341e+06,0.000000,0.000000,0.000000,2.777268e+07,3.906010e+08,7.540000e+05,7.355000e+05,7.900000e+05,7.990000e+05,...,-3.255000e+05,4.137250e+06,2.212500e+06,5.000000e+04,2.480000e+05,8.675000e+04,1.395000e+05,4.000000e+04,2.075000e+05,1.000000e+04
50%,7.876530e+06,0.000000,0.000000,0.000000,4.236137e+07,3.906010e+08,4.821000e+06,5.002000e+06,4.444000e+06,4.695000e+06,...,1.150000e+04,3.864950e+07,2.950600e+07,4.260000e+05,1.141000e+06,4.170000e+05,5.760000e+05,1.610000e+05,9.740000e+05,9.500000e+04
75%,9.985408e+06,0.000000,0.000000,0.000000,7.111622e+07,3.912010e+08,2.046200e+07,2.242800e+07,1.891800e+07,2.078800e+07,...,1.202500e+06,2.480088e+08,2.251600e+08,2.642000e+06,4.576000e+06,2.062500e+06,2.049500e+06,5.000000e+05,4.080500e+06,6.530000e+05
max,1.254730e+07,1.000000,1.000000,1.000000,9.971510e+07,7.708010e+08,1.062454e+10,5.336259e+10,1.061781e+10,5.322594e+10,...,1.355626e+09,9.091172e+10,9.100927e+10,3.553420e+08,6.939350e+08,2.043270e+08,3.110270e+08,6.589000e+06,7.052140e+08,4.644630e+08


In [23]:
data.drop(['2019, Поступления от выпуска акций, увеличения долей участия, RUB',
           '2019, Поступления от выпуска облигаций, векселей и других долговых ценных бумаг и др., RUB',
           '2019, Поступления от перепродажи финансовых вложений, RUB',
           '2019, Приобретение акций (долей участия), RUB',
           '2018, Платежи по процентам, RUB',
           '2019, Выкуп акций (долей) у собственников, RUB',
           '2019, Платежи по процентам, RUB',
           '2019, Продажа акций других организаций, RUB',
           '2019, Собственные акции, выкупленные у акционеров, RUB',
           '2019, Доходы от участия в других организациях, RUB',
           '2019, Поступление денежных вкладов собственников (участников), RUB',
           '2019, Доходные вложения в материальные ценности, RUB',
           '2019, Продажа внеоборотных активов, RUB',
           '2019, Прибыль от приносящей доход деятельности, RUB',
           '2019, Приобретение основных средств, инвентаря и иного имущества, RUB',
           '2019, Получение дивидендов, процентов, RUB',
           '2019, Переоценка внеоборотных активов, RUB',
           '2019, Выплата дивидендов, RUB',
           '2019, Поступления арендных, лицензионных, комиссионных платежей, роялти, RUB',
           '2019, Доходы будущих периодов , RUB',
           '2019, Нематериальные активы, RUB',
           '2019, Резервный капитал, RUB',
           '2019, Прочие внеоборотные активы, RUB',
           '2019, Приобретение внеоборотных активов, RUB',
           '2019, Расходы на целевые мероприятия, RUB',
           '2019, Приобретение долговых ценных бумаг, предоставление займов, RUB',
           '2019, Поступления от возврата займов, продажи долговых ценных бумаг, RUB',
           '2019, Оплата процентов по долговым обязательствам, RUB',
           '2019, Отложенные налоговые обязательства, RUB',
           '2019, Расходы на содержание аппарата управления, RUB',
           '2019, Поступления от инвестиционных операций, RUB',
           '2019, Остаток средств на конец отчетного года, RUB',
           '2018, Платежи по инвестиционным операциям, RUB', 
           '2018, Прочие долгосрочные обязательства, RUB',
           '2018, Погашение векселей, возврат кредитов и займов, RUB', 
           '2019, Платежи по инвестиционным операциям, RUB', 
           '2019, Прочие краткосрочные обязательства, RUB', 
           '2019, Всего поступило средств, RUB',
           '2019, Отложенные налоговые активы, RUB',
           '2019, Всего использовано средств, RUB',
           '2019, Прочие долгосрочные обязательства, RUB',
           '2018, Прочие краткосрочные обязательства, RUB',
           '2019, Сальдо денежных потоков от инвестиционных операций, RUB',
           '2019, Погашение векселей, возврат кредитов и займов, RUB',
           '2018, Получение кредитов и займов, RUB', 
           '2019, Проценты к получению, RUB', 
           '2019, Налог на прибыль, RUB', 
           '2019, Добавочный капитал, RUB', 
           '2019, Получение кредитов и займов, RUB', 
           '2019, Платежи по финансовым операциям, RUB',
           '2019, Поступления от финансовых операций, RUB',
           '2019, НДС по приобретенным ценностям , RUB',
           '2019, Долгосрочные финансовые вложения, RUB',
           '2019, Сальдо денежных потоков от финансовых операций, RUB', 
           '2017, Налог на имущество, RUB',
           '2019, Краткосрочные финансовые вложения, RUB'
          ], axis='columns', inplace=True)

In [28]:
pd.isnull(data).sum().nlargest(50)

2018, Налог на имущество, RUB                              11885
2018, Оплата труда, RUB                                    11885
2018, Проценты к уплате, RUB                               11880
2019, Прочие оборотные активы, RUB                         11860
2019, Поступления от продаж, RUB                           11858
2019, Проценты к уплате, RUB                               11856
2019, Коммерческие расходы, RUB                            11845
2019, Оплата труда, RUB                                    11819
2018, Коммерческие расходы, RUB                            11789
2019, Поступления от текущих операций, RUB                 11764
2019, Платежи поставщикам и подрядчикам, RUB               11762
2019, Сальдо денежных потоков за отчетный период, RUB      11730
2019, Поступление денежных средств всего, RUB              11724
2019, Сальдо денежных потоков от текущих операций, RUB     11723
2019, Платежи по текущим операциям, RUB                    11706
2019, Платежей всего, RUB